In [1]:
# Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import requests

from scipy.optimize import minimize
import scipy.stats as stats
from statsmodels.base.model import GenericLikelihoodModel

#-----------------------------------------------------------------------------------------------------------------------------
#   IMPORTING THE DATA I WILL USE
#-----------------------------------------------------------------------------------------------------------------------------

# Dataframe containing how every representative voted in each rollcall.
# I will focus (for now) just in the congress number 45.
dtavotes = pd.read_csv('Hall_votes.csv')
dtavotes = dtavotes[(dtavotes.congress == 45) & (dtavotes.rollnumber < 100)]

# Dataframe containing data on political characteristics of the representative.
# I will focus (for now) just in the congress number 45.
dtamembers = pd.read_csv('HSall_members.csv')
dtamembers = dtamembers[(dtamembers.congress == 45) & (dtamembers.chamber=="House")]
dtamembers = dtamembers[['congress' , 'icpsr' , 'chamber' , 'party_code', 'nominate_dim1' , 'nominate_dim2' ]]

# Merging votes andpolitical characteristics of representatives 
dtavotesandmembers = pd.merge(dtavotes, dtamembers, how="left", on=['congress' , 'icpsr' , 'chamber'])

# cast_code represent how did they vote for every rollcall. 
# I will create a variable named yeah if the representive voted in favor of the bill.
dtavotesandmembers['yeah']=np.where(dtavotesandmembers.cast_code== 1 , True, False)
dtavotesandmembers['yeah']=np.where(dtavotesandmembers.cast_code== 2 , True, dtavotesandmembers.yeah)
dtavotesandmembers['yeah']=np.where(dtavotesandmembers.cast_code== 3 , True, dtavotesandmembers.yeah)

In [2]:
#-----------------------------------------------------------------------------------------------------------------------------
#   CANDIDATES AND ROLLCALL ideology
#-----------------------------------------------------------------------------------------------------------------------------

# Creating a dataframe containing the ideology of every candidate 
# per chamber (only low chamber for now) per congress(only 45 for now)
MEMBERSVAL=dtamembers[['congress' , 'icpsr' , 'chamber']]

# Creating a dataframe containing the ideology of every candidate 
# per chamber (only low chamber for now) per congress(only 45 for now)
# In this dataframe, I start using a dataframe called "dtavotes", which 
# contain a higher dimmention (congress x rollcall x icpsr(representative) x chamber)
# That´s why I drop duplicates
VOTEYESVAL=dtavotes[['congress' , 'rollnumber' , 'chamber']]
VOTEYESVAL=VOTEYESVAL.drop_duplicates()
VOTENAHVAL=dtavotes[['congress' , 'rollnumber' , 'chamber']]
VOTENAHVAL=VOTEYESVAL.drop_duplicates()


# THESE ARE FAKE VALUES !!!!! the objective is come up with the right values in the optimization
# I set this fake values because the program requirres to start in an arbitrary value.
MEMBERSVAL['ideology']=0.5
VOTEYESVAL['rollvalue']=0.3
VOTENAHVAL['rollnovalue']=0.2


<ipython-input-2-264910cc73bd>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MEMBERSVAL['ideology']=0.5


**Optimization**

Here I set, for each representative, a utility level that depend on her ieology and the ideology charge of the rollcall (value, from now on).

The Utility function has the following expression: $U_{ij}=(\rho_{j}-\theta_{i})^{2} + \epsilon_{ij}$

The candidate will aprove the call with probability: 
$Pr((\rho_{j}-\theta_{i})^{2} + \epsilon_{ij} > (v_{j}-\theta_{i})^{2} + \varepsilon_{ij})$
$=Pr((\rho_{j}-\theta_{i})^{2} - (v_{j}-\theta_{i})^{2} >  \varepsilon_{ij}-\epsilon_{ij})$

So the Likelyhoof function should be:

$\prod_{i}^{I} \prod_{t}^{T} \phi((\rho_{j}-\theta_{i})^{2})^{Yes_{i}} (1-\phi((\rho_{j}-\theta_{i})^{2}))^{1-Yes_{i}} $


In [3]:

def MLEutility(ideal, value, novalue):

    # Plug the candidate parameters for ideology
    for i in range(0,len(MEMBERSVAL)):
        MEMBERSVAL.loc[ i , 'ideology' ] = ideal[0,i]

    # Plug the candidate parameters for ideological value of voting yes
    VOTEYESVAL=VOTEYESVAL.reset_index(drop=True)
    for i in range(0,len(VOTEYESVAL)):
        VOTEYESVAL.loc[ i , 'rollvalue' ] = value[0,i]
    
    # Plug the candidate parameters for ideological value of "not voting yes"
    VOTENAHVAL=VOTENAHVAL.reset_index(drop=True)
    for i in range(0,len(VOTENAHVAL)):
        VOTENAHVAL.loc[ i , 'rollnovalue' ] = novalue[0,i]   
    
    # Create a dataframe (called dtavotesandmembers0) with the ideology scores
    dtavotesandmembers2 = pd.merge(dtavotesandmembers, MEMBERSVAL, how="left", on=['congress' , 'icpsr' , 'chamber'])
    dtavotesandmembers3 = pd.merge(dtavotesandmembers2, VOTEYESVAL, how="left", on=['congress' , 'rollnumber' , 'chamber'])
    dtavotesandmembers0 = pd.merge(dtavotesandmembers3, VOTENAHVAL, how="left", on=['congress' , 'rollnumber' , 'chamber'])
    # Calculate Utility function
    dtavotesandmembers0['Utility']=np.abs(dtavotesandmembers0.rollvalue-dtavotesandmembers0.ideology)**2-np.abs(dtavotesandmembers0.rollnovalue-dtavotesandmembers0.ideology)**2
    # Calculating probability of voting yes or not given the parameters
    dtavotesandmembers0['probyeah']=stats.norm.pdf(dtavotesandmembers0.Utility, loc=0, scale=1)
    dtavotesandmembers0['probneah']=1-stats.norm.pdf(dtavotesandmembers0.Utility, loc=0, scale=1)
    # Calculating (the negative value of) the log likelihood.
    # Negative value because I am going to minimize it (instead of maximizinf LL)
    dtavotesandmembers0['like']=(dtavotesandmembers0.probyeah*(dtavotesandmembers0.yeah)+dtavotesandmembers0.probneah*(1-dtavotesandmembers0.yeah))
    dtavotesandmembers0['loglike']=np.log(dtavotesandmembers0.like)
    negLL = -np.sum(dtavotesandmembers0.loglike)

    return(negLL)



In [4]:
I = MEMBERSVAL['ideology']
idealfirst = np.matrix(I, dtype=float)
V = VOTEYESVAL['rollvalue']
valuefirst = np.matrix(V, dtype=float)
NV = VOTENAHVAL['rollnovalue']
novaluefirst = np.matrix(NV, dtype=float)


In [5]:
results = minimize(MLEutility, idealfirst, valuefirst, novaluefirst , method = 'Nelder-Mead', options={'disp': True})

TypeError: minimize() got multiple values for argument 'method'

**I get an error in this finnal part of the code!!!**